In [8]:
import torch.nn as nn
import torch

In [10]:
training_data = [
    ('The cat ate the fish'.split(), ['DET', 'NN', 'V', 'DET', 'NN']),
    ('They read that book'.split(), ['NN', 'V', 'DET', "NN"])
]

testing_data = [('They ate the fish'.split())]

#构建每个单词的索引字典
word2idx = {}
for sent, tags in training_data:
    print(sent)
    for word  in sent:
        if word not in word2idx:
            word2idx[word] = len(word2idx)

print(word2idx)

#手工设置词性的索引字典
tag2idx = {'DET':0, 'NN':1, "V":2}

#构建网路
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return(torch.zeros(1, 1, self.hidden_dim))
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(sentence, 
                                                      1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence)))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

['The', 'cat', 'ate', 'the', 'fish']
['They', 'read', 'that', 'book']
{'The': 0, 'cat': 1, 'ate': 2, 'the': 3, 'fish': 4, 'They': 5, 'read': 6, 'that': 7, 'book': 8}


In [ ]:
def prepare_sequence(seq, to_idx):
    idxs = [to_idx[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return tensor

In [ ]:
#训练网络
EMBEDDING_DIM=10
HIDDEN_DIM=3
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM)